In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

os.chdir('..')

In [3]:
from collections import defaultdict
from sklearn.model_selection import train_test_split

from fgclassifier.utils import read_data, get_dataset

X_train, y_train = read_data(get_dataset('train_en'), flavor=None)
X_test, y_test = read_data(get_dataset('valid_en'), flavor=None)

2018-12-03 21:50:11,147 [INFO] Reading data/english_train.csv..
2018-12-03 21:50:11,375 [INFO] Reading data/english_valid.csv..


In [ ]:
fm = defaultdict(dict)

In [ ]:
from fgclassifier.features import FeaturePipeline, logger

for name in ['word2vec_en', 'word2vec_en_minmax']:
    logger.info(f'Building features for {name}...')
    model = FeaturePipeline.from_spec(name, cache=fm)
    model.fit_transform(X_train)
    model.transform(X_test)

C:\Users\jyyjc\Anaconda3\envs\idp\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
2018-12-03 21:50:12,600 [INFO] 'pattern' package not found; tag filters are not available for English
2018-12-03 21:50:12,627 [INFO] Building features for word2vec_en...
2018-12-03 21:50:30,054 [INFO] collecting all words and their counts
2018-12-03 21:50:30,056 [INFO] PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-12-03 21:50:30,742 [INFO] collected 23515 word types from a corpus of 1877982 raw words and 8000 sentences
2018-12-03 21:50:30,744 [INFO] Loading a fresh vocabulary
2018-12-03 21:50:30,887 [INFO] effective_min_count=3 retains 10128 unique words (43% of original 23515, drops 13387)
2018-12-03 21:50:30,890 [INFO] effective_min_count=3 leaves 1861693 word corpus (99% of original 1877982, drops 16289)
2018-12-03 21:50:30,983 [INFO] deleting th

2018-12-03 21:51:07,443 [INFO] EPOCH 10 - PROGRESS: at 82.66% examples, 346100 words/s, in_qsize 6, out_qsize 0
2018-12-03 21:51:07,966 [INFO] worker thread finished; awaiting finish of 2 more threads
2018-12-03 21:51:07,979 [INFO] worker thread finished; awaiting finish of 1 more threads
2018-12-03 21:51:07,981 [INFO] worker thread finished; awaiting finish of 0 more threads
2018-12-03 21:51:07,983 [INFO] EPOCH - 10 : training on 1877982 raw words (1291265 effective words) took 3.6s, 357866 effective words/s
2018-12-03 21:51:07,986 [INFO] training on a 18779820 raw words (12913869 effective words) took 36.5s, 354134 effective words/s


In [ ]:
scores_train, avg_score_train = defaultdict(dict), defaultdict(dict)
scores_test, avg_score_test = defaultdict(dict), defaultdict(dict)

In [ ]:
from fgclassifier.train import fm_cross_check
from IPython.display import clear_output

In [ ]:
conf = {
    'fm_cache': fm,
    'X_train': X_train,
    'X_test': X_test,
    'y_train': y_train,
    'y_test': y_test,
    'results': {
        'models': {},
        'avg': avg_score_test,
        'all': scores_test,
        'avg_train': avg_score_train,
        'all_train': scores_train,
    }
}

# All other models can run on many classifiers
results = fm_cross_check(
    ['word2vec_en_minmax'],
    ['ComplementNB', 'QDA', 'RBF', 'SVM_Poly2', 'SVM_Poly3', 'LDA',
     'LinearSVC', 'Ridge', 'Logistic'], **conf)

clear_output()

In [ ]:
rows = {}
for fm_name in all_scores:
    for clf_name in all_scores[fm_name]:
        key = f'{fm_name}.{clf_name}'
        rows[key] = [all_avg_scores[fm_name][clf_name],
                     *all_scores[fm_name][clf_name]]
df = pd.DataFrame(rows)
df.index = ['average', *y_train.columns]
df = df.T.sort_values('average', ascending=False)
df

In [10]:
import numpy as np

print(np.mean(scores))

0.40654159090857
